<a href="https://colab.research.google.com/github/pdh21/OCF_PVmapper/blob/main/Example_DeepNet.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

![](https://avatars.githubusercontent.com/u/48357542?s=200&v=4)

## PV mapping Example:
[Peter Hurley](http://pdh21.github.io/) 

In this notebook, we will give a simple example of training a deep net to detect solar panels from remote sensing data. 

In [ ]:
from google.colab import drive

drive.mount(‘/content/gdrive’)